# MinViT

In this notebook, I attempt to explain the vision transformer (ViT) architecture, which has found its way into computer vision as a powerful alternative to Convolutional Neural Networks (CNNs).

This implementation will focus on classifying the CIFAR-10 dataset, but is adaptable to many tasks, including semantic segmentation, instance segmentation, and image generation.

We begin by downloading the CIFAR-10 dataset, and transforming the data to `torch.Tensor`s.

In [17]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms

import math

transform = transforms.Compose([transforms.ToTensor()])

train_data = datasets.CIFAR10(root='./data/cifar-10', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data/cifar-10', train=False, download=True, transform=transform)

train_data, test_data

Files already downloaded and verified
Files already downloaded and verified


(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: ./data/cifar-10
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
            ),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: ./data/cifar-10
     Split: Test
     StandardTransform
 Transform: Compose(
                ToTensor()
            ))

The images are represented as 3 channel (RGB) 32x32 pixel images. The dataset can be indexed, with the first index being the image index, and the second index indexing either the image data or the target. The pixel values are represented as `torch.float32` values from 0 to 1.

In [48]:
train_data.data.shape, len(train_data.targets)

((50000, 32, 32, 3), 50000)

In [49]:
train_data[0][0].numpy().shape, train_data[0][1]

((3, 32, 32), 6)

In [54]:
train_data[0][0][0], train_data[0][0].dtype

(tensor([[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
         [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
         [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
         ...,
         [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
         [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
         [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]]),
 torch.float32)

If you are familiar with the transformer architecture, you likely know that transformers work with vectors to model different modalities. For a text-based modality, this means somehow tokenizing a string of text into characters or larger chunks, and training an embedding table to represent each token as a vector. As an example, the string "This is a test." may tokenize as follows:

<span style="background-color:rgba(107,64,216,.3);white-space:pre;">This</span><span style="background-color:rgba(104,222,122,.4);white-space:pre;"> is</span><span style="background-color:rgba(244,172,54,.4);white-space:pre;"> a</span><span style="background-color:rgba(239,65,70,.4);white-space:pre;"> test</span><span style="background-color:rgba(39,181,234,.4);white-space:pre;">.</span>

To adapt the transformer architecture for image tasks, we need to represent image data as a sequence a vectors, similar to how text is tokenized. In the original [ViT paper](https://arxiv.org/abs/2010.11929), the authors address this by dividing an image into many patches and flattening them into vectors. With CIFAR-10, an image $x \in \mathbb{R}^{H\times W\times C}$ is turned into several flattened 2D patches of the form $x_p \in \mathbb{R}^{N\times (P^2\cdot C)}$, where $(H,W)$ are the image dimensions (32x32), $C$ is the number of channels (3 for RGB), and $P$ is the patch size. The number of flattened 2D patches is then $N = \frac{HW}{P^2}$. Finally, we project the flattened patches to latent vectors of size $D$, using the linear projection $\mathbf{E} \in \mathbb{R}^{(P^2\cdot C)\times D}$.

In [55]:
patch_size = 4
for i in range(0, 32, patch_size):
    for j in range(0, 32, patch_size):
        patch = train_data[0][0][:, i:i+patch_size, j:j+patch_size]

print(patch.shape)

torch.Size([3, 4, 4])


A naive implementation of turning the images into patches may look like the code above. However, we can accelerate this process by using `torch.Tensor.unfold`.

In [165]:
image = torch.arange(0.,48).reshape(1, 3, 4, 4) # batch size, channels, width, height
image, image.unfold(2, 2, 2).unfold(3, 2, 2) # first unfold width, then height into 2x2 patches

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]],
 
          [[16., 17., 18., 19.],
           [20., 21., 22., 23.],
           [24., 25., 26., 27.],
           [28., 29., 30., 31.]],
 
          [[32., 33., 34., 35.],
           [36., 37., 38., 39.],
           [40., 41., 42., 43.],
           [44., 45., 46., 47.]]]]),
 tensor([[[[[[ 0.,  1.],
             [ 4.,  5.]],
 
            [[ 2.,  3.],
             [ 6.,  7.]]],
 
 
           [[[ 8.,  9.],
             [12., 13.]],
 
            [[10., 11.],
             [14., 15.]]]],
 
 
 
          [[[[16., 17.],
             [20., 21.]],
 
            [[18., 19.],
             [22., 23.]]],
 
 
           [[[24., 25.],
             [28., 29.]],
 
            [[26., 27.],
             [30., 31.]]]],
 
 
 
          [[[[32., 33.],
             [36., 37.]],
 
            [[34., 35.],
             [38., 39.]]],
 
 
           [[[40., 41.],
             [44.,

We can then view this tensor of 2D patches as 1D vectors using `Tensor.reshape`.

In [199]:
image.unfold(2,2,2).unfold(3,2,2).reshape(-1, 4)



tensor([[ 0.,  1.,  4.,  5.],
        [ 2.,  3.,  6.,  7.],
        [ 8.,  9., 12., 13.],
        [10., 11., 14., 15.],
        [16., 17., 20., 21.],
        [18., 19., 22., 23.],
        [24., 25., 28., 29.],
        [26., 27., 30., 31.],
        [32., 33., 36., 37.],
        [34., 35., 38., 39.],
        [40., 41., 44., 45.],
        [42., 43., 46., 47.]])

Another embedding that is critical for the transformer architecture to understand context is the positional embedding. In the text modality, the positional embedding is often implemented using cosine functions. To represent 2D position however, we will use a standard embedding table that is learned during training. There are other possibilities for representing position, including 2D-aware positional embeddings, but these are harder to implement and result in negligble performance differences.

Finally, we also want to use a specific vector for information about the class of the image to the transformer input. Through each transformer block, this vector is modified until the end, when it is fed into a multi-layer perceptron to determine the class of the image. We will use a learnable embedding for this and prepend it to the other embedded patches.

In [208]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size = 32, patch_size = 4, in_chans = 3, embed_dim = 768):
        super().__init__()
        self.img_size   = img_size
        self.patch_size = patch_size    # P
        self.in_chans   = in_chans      # C
        self.embed_dim  = embed_dim     # D

        self.num_patches = (img_size // patch_size) ** 2        # N = H*W/P^2
        self.flatten_dim = patch_size * patch_size * in_chans   # P^2*C
        
        self.proj = nn.Linear(self.flatten_dim, embed_dim) # (P^2*C,D)

        self.position_embed = nn.Parameter(torch.zeros(1, 1 + self.num_patches, embed_dim))
        self.class_embed    = nn.Parameter(torch.zeros(1, 1, embed_dim))

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        x = x.contiguous().view(B, C, self.num_patches, -1)
        x = x.permute(0, 2, 1, 3).reshape(B, self.num_patches, -1)

        x = self.proj(x)

        cls_emb = self.class_embed.expand(B, -1, -1)
        x = torch.cat((cls_emb, x), dim = 1)

        x = x + self.position_embed
        return x

In [296]:
patch_embed = PatchEmbedding()

embeddings = patch_embed(torch.stack([train_data[i][0] for i in range(10)]))

embeddings, embeddings.shape

(tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.0569, -0.0865,  0.0411,  ...,  0.1318,  0.1935, -0.0940],
          [ 0.0846,  0.1228, -0.0564,  ...,  0.0554,  0.4496,  0.0409],
          ...,
          [ 0.0518,  0.1557, -0.0584,  ...,  0.1239,  0.3988,  0.0172],
          [ 0.0330,  0.0413, -0.0276,  ...,  0.1904,  0.3197,  0.0385],
          [ 0.0961,  0.1149, -0.0723,  ...,  0.2575,  0.2643,  0.0029]],
 
         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0936,  0.1547,  0.0363,  ...,  0.2752,  0.4989, -0.0243],
          [-0.0062,  0.1697,  0.2444,  ...,  0.2540,  0.8188, -0.0052],
          ...,
          [-0.0646, -0.0800,  0.0320,  ...,  0.2009,  0.2252, -0.0501],
          [-0.0562, -0.0649,  0.1433,  ...,  0.2242,  0.4622, -0.0627],
          [-0.0676, -0.0061,  0.0631,  ...,  0.1936,  0.4408, -0.0672]],
 
         [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.2059,  0.3761,

As seen above, we are able to embed batches into our desired embedding dimension, with a correct number of vectors $N+1$.

We can now continue to implement the standard transformer architecture, with one notable change from GPT-like architectures. The attention mechanism of GPT involves multi-headed **causal** self-attention, which means that vectors are only allowed to query and interact with previous vectors. Although this makes sense in a language model that wants to extract causal contextual information, we want all vectors to communicate with all other vectors, and want to prevent applying an attention mask. Otherwise, the implementation remains unchanged.

In [219]:
class MutliheadedSelfAttention(nn.Module):
    def __init__(self, embed_dim = 768, num_heads = 4, bias = False, dropout=0.1):
        super().__init__()
        assert embed_dim % num_heads == 0

        self.embed_dim   = embed_dim
        self.num_heads   = num_heads
        self.head_dim    = embed_dim // num_heads

        self.query   = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.key     = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.value   = nn.Linear(embed_dim, embed_dim, bias=bias)

        self.out     = nn.Linear(embed_dim, embed_dim)

        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, N, _ = x.size()

        q = self.query(x).view(B, N, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        k = self.key(x).view(B, N, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        v = self.value(x).view(B, N, self.num_heads, self.head_dim).permute(0, 2, 1, 3)

        # do NOT use causal attention as we are not dealing with sequential data (image patches are unordered)
        attn = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(self.head_dim))
        attn = attn.softmax(dim=-1)
        
        #y = torch.nn.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout=self.attn_dropout if self.training else 0, is_causal=False)
        #y = y.transpose(1, 2).contiguous().view(B, N, C)
        #y = self.resid_dropout(self.c_proj(y))

        out = (attn @ v).permute(0, 2, 1, 3).contiguous().view(B, N, self.embed_dim)

        out = self.out(out)

        return out

In [274]:
MSA = MutliheadedSelfAttention()
LN = nn.LayerNorm(embeddings.shape, bias=False)

MSA(LN(embeddings))

tensor([[[-0.1757, -0.2464, -0.4134,  ..., -0.3564,  0.0455,  0.1032],
         [-0.1715, -0.2462, -0.4092,  ..., -0.3610,  0.0515,  0.1003],
         [-0.1724, -0.2474, -0.4128,  ..., -0.3645,  0.0555,  0.1002],
         ...,
         [-0.1710, -0.2473, -0.4098,  ..., -0.3666,  0.0564,  0.0996],
         [-0.1696, -0.2475, -0.4091,  ..., -0.3672,  0.0566,  0.0992],
         [-0.1683, -0.2473, -0.4068,  ..., -0.3709,  0.0577,  0.0993]],

        [[-0.1521, -0.2388, -0.6402,  ..., -0.4138,  0.0234,  0.1322],
         [-0.1376, -0.2526, -0.6332,  ..., -0.4497,  0.0539,  0.1275],
         [-0.1403, -0.2566, -0.6437,  ..., -0.4529,  0.0592,  0.1265],
         ...,
         [-0.1418, -0.2423, -0.6297,  ..., -0.4284,  0.0398,  0.1270],
         [-0.1387, -0.2435, -0.6282,  ..., -0.4340,  0.0466,  0.1253],
         [-0.1456, -0.2463, -0.6343,  ..., -0.4366,  0.0401,  0.1279]],

        [[-0.1408, -0.2385, -0.6398,  ..., -0.4515,  0.0103,  0.1567],
         [-0.0718, -0.3082, -0.6767,  ..., -0

Finally, we want to implement the multi-layer perceptron and combine all our modules into the transformer block.

In [297]:
class MLP(nn.Module):
    def __init__(self, embed_dim = 768, bias = False, dropout = 0.1):
        super().__init__()
        self.c_fc = nn.Linear(embed_dim, embed_dim * 4, bias=bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(embed_dim * 4, embed_dim, bias=bias)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)

        return x
    
class Block(nn.Module):

    def __init__(self, embed_dim = 768, bias = False):
        super().__init__()
        self.ln_1 = nn.LayerNorm(embed_dim, bias=bias)
        self.attn = MutliheadedSelfAttention()
        self.ln_2 = nn.LayerNorm(embed_dim, bias=bias)
        self.mlp = MLP()

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x
    
    

The final element to consider in our model is the output. Unlike a transformer model like GPT, we would like to produce a probability distribution of the various image classes in CIFAR-10. To achieve this, we use the class vector prepended to the input that was mentioned earlier. After the vector has passed through each transformer block, we can finally take the class vector and pass it through a linear projection once more to get a probability distribution across all ten image classes.

In [307]:
class ViT(nn.Module):

    def __init__(self, embed_dim = 768, num_layers = 4, bias = False):
        super().__init__()

        self.transformer = nn.ModuleDict(dict(
            pe = PatchEmbedding(),
            drop = nn.Dropout(0.1),
            h = nn.ModuleList([Block() for _ in range(num_layers)]),
            ln_f = nn.LayerNorm(embed_dim)
        ))
        self.head = nn.Linear(embed_dim, 10, bias=False)


        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self):
        n_params = sum(p.numel() for p in self.parameters())
        return n_params

    def forward(self, x, targets=None):
        emb = self.transformer.pe(x)
        x = self.transformer.drop(emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        class_token = x[:, 0]
        logits = self.head(class_token)
        return logits


In [308]:
vit = ViT()
vit(torch.stack([train_data[i][0] for i in range(10)]))

number of parameters: 28.42M


tensor([[-0.6155,  0.0833,  0.3612,  0.1262,  0.3042,  0.1746, -0.8091, -0.1743,
          0.7834, -0.0631],
        [-1.0184, -0.1414, -0.1417, -0.2963,  0.3737,  0.0205, -0.6493, -0.0109,
          0.8699, -0.1459],
        [-0.9966,  0.1542, -0.1182, -0.0219,  0.5765,  0.0459, -0.4171, -0.1138,
          0.9797, -0.0116],
        [-0.7114,  0.0771,  0.1259,  0.0709,  0.4262,  0.2157, -0.9031, -0.0468,
          0.7561, -0.0685],
        [-0.7684,  0.0140, -0.1220, -0.2881,  0.6543, -0.0601, -0.4210, -0.1563,
          0.8449,  0.0266],
        [-0.6628,  0.2121, -0.0296,  0.1316,  0.4504,  0.5865, -1.1234, -0.1563,
          0.6179,  0.0671],
        [-0.8797,  0.1741,  0.2067, -0.1092,  0.4820,  0.0855, -0.3633,  0.1939,
          0.8685, -0.4827],
        [-0.8408,  0.1524, -0.1209, -0.2138,  0.3352,  0.1860, -0.5956,  0.1675,
          0.9737,  0.0405],
        [-0.8931, -0.3047, -0.1372, -0.3863,  0.4625,  0.2419, -0.4026,  0.1199,
          0.7785,  0.1355],
        [-0.4177,  